# Assignment 4
In Assignment 4, we consider a random coefficient model **with the unobserved fixed effects.** 
- Description of Assignment 4: [link](https://kohei-kawaguchi.github.io/EmpiricalIO/assignment4.html)

In [1]:
# import packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
from numba import jit, njit, prange
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# 1 Simulate data

## 1.1 Set the seed, constants, and parameters of interest.

In [3]:
# set the seed
np.random.seed(1019)
# number of products
J = 10
# dimension of product characteristics including the intercept (=1)
K = 3
# number of markets
T = 20


# number of consumers per market
N = 100
# number of Monte Carlo
L = 100

- $[\beta_{01}, \cdots, \beta_{0K}]$ is referred to as beta.
- $[\sigma_{1}, \cdots, \sigma_{K}]$ is referred to as sigma.

In [4]:
# set parameters of interest
beta = np.random.standard_normal(size = (K,1))
beta[0] = 4

print('beta: ', beta)

sigma = abs(np.random.standard_normal(size = (K,1))) 
print('sigma: ', sigma)

mu = 0.5
omega = 1

beta:  [[ 4.        ]
 [-0.34712881]
 [-0.64393574]]
sigma:  [[0.62307216]
 [1.0996384 ]
 [1.01118079]]


In [5]:
# set auxiliary parameters
price_xi = 1
sd_x = 2
sd_xi = 0.5
sd_c = 0.05
sd_p = 0.05

## 1.2 X
X is the data frame such that a row contains the characteristics vector $x_j$ of a product and columns are product index and observed product characteristics. The dimension of the characteristics $K$ is specified above. Add the row of the outside option whose index is 0 and all the characteristics are zero.

- The product-market characteristics: $$x_{j1}=1, x_{jk}\sim N(0, \sigma_x), k=2,\cdots,K,$$ where $\sigma_x$ is referred to as sd_x in the code.

In [6]:
# X product-market characteristics
X = np.random.normal(scale = sd_x, size = (J+1,K-1))
X = pd.DataFrame(X, columns = ['x_2', 'x_3'])
X['x_1'] = 1
X['j'] = X.index
X.loc[X['j'] == 0, ['x_1', 'x_2', 'x_3']] = 0

print('shape of X: ', X.shape)
X.head()

shape of X:  (11, 4)


,x_2,x_3,x_1,j
0,0.000000,0.000000,0,0
1,3.158113,0.468290,1,1
2,0.507840,3.493566,1,2
3,1.896881,-0.235369,1,3
4,-0.651372,-0.613362,1,4


## 1.3 M
M is the data frame such that a row contains the product-market specific fixed effect $\xi_{jt}$, marginal cost $c_{jt}$, and price $p_{jt}$. **In this assignment, we drop products in a different way from Assignment 3. In order to change the number of available products in each market, for each market, first draw $J_t$ from a discrete uniform distribution between 1 and $J$**. The variation in the available products is important for the identification of the distribution of consumer-level unobserved heterogeneity. Add the row of the outside option to each market whose index is 0 and all the variables take value zero.

- We draw $\xi_{jt}$ from i.i.d. normal distribution with mean 0 and standard deviation $\sigma_{\xi}$: $$\xi_{jt}\sim Normal(0, \sigma_{\xi})$$
  - $\xi_{jt}$ is product-market specific fixed effect. $p_{jt}$ can be correlated with $\xi_{jt}$ but $x_{jt}$s are independent of $\xi_{jt}$.

- The marginal cost of product $j$ in market $t$: $$c_{jt}\sim logNormal(0, \sigma_c),$$ where $\sigma_c$ is referred to as sd_c in the code.
- The retail price: $$p_{jt}-c_{jt} \sim logNormal(\gamma \xi_{jt}, \sigma_p),$$ where $\gamma$ is referred to as price_xi, and $\sigma_p$ as sd_p in the code. This price is not the equilibrium price.

In [7]:
# j = 1, 2, ..., J; t = 1, 2, ..., T
M = pd.DataFrame([(j+1,1) for j in range(J)], columns = ['j', 'temp'])\
.merge(pd.DataFrame([(t+1,1) for t in range(T)], columns = ['t', 'temp']), how = 'outer')
M.drop(columns = 'temp', inplace =True)

M['xi'] = np.random.normal(scale = sd_xi, size = (len(M),1))
M['c'] = np.random.lognormal(sigma = sd_c, size = (len(M),1))
M['p'] = M.apply(lambda x: np.random.lognormal(mean = price_xi * x.xi, sigma = sd_p), axis = 1)
M['p'] = M['p'] + M['c']

M.sort_values(by = ['t', 'j'], inplace = True)

# for each market, draw 𝐽𝑡 from a discrete uniform distribution between 1 and  𝐽
temp2 = pd.DataFrame()
for t in M.t.unique():
    J_t = np.random.randint(1, J)
    temp1 = M[M.t == t].sample(n = J_t, random_state = 230)
    temp2 = pd.concat([temp1, temp2])
    
# add the row of the outside option to each market whose index is 0 and all the variables take value zero.
# M.loc[M.j == 0, ['xi','c', 'p']] = 0
temp3 = M[['t']].drop_duplicates()
for col in ['j', 'xi', 'c', 'p']:
    temp3[col] = np.zeros(shape = (len(temp3), 1))

M = pd.concat([temp2, temp3])
print('shape of M: ', M.shape)
M.head()

shape of M:  (126, 5)


,j,t,xi,c,p
119,6.0,20,-0.834950,1.050409,1.485568
118,6.0,19,0.558018,0.953264,2.764732
98,5.0,19,0.392971,0.957834,2.281137
158,8.0,19,0.062806,1.072073,2.085452
138,7.0,19,-0.570439,1.069503,1.638399


## 1.4 V
Generate the consumer-level heterogeneity. V is the data frame such that a row contains the vector of shocks to consumer-level heterogeneity, $(v_i', v_i)$ . They are all i.i.d. standard normal random variables.

- $\beta_{itk} = \beta_{0k} + \sigma_k v_{itk},$ where $v_{itk}$ for $k=1,2,\cdots, K$ are i.i.d. standard normal random variables.
  
- $\alpha_{it} = -exp(\mu + \omega v_{it}) = -exp(\mu + \omega^2/2) + [-exp(\mu + \omega v_{it}) + exp(\mu + \omega^2/2)] = \alpha_0 + \tilde{\alpha}_{it},$ where $v_{it}$ are i.i.d. standard normal random variables.

In [8]:
V = pd.DataFrame([(i+1,1) for i in range(N)], columns = ['i', 'temp'])\
.merge(pd.DataFrame([(t+1,1) for t in range(T)], columns = ['t', 'temp']), how = 'outer')
V.drop(columns = 'temp', inplace =True)
for col in ['v_x_1', 'v_x_2', 'v_x_3', 'v_p']:
    V[col] = np.random.normal(size = (N*T, 1))

print('shape of V: ', V.shape)
V.head()

shape of V:  (2000, 6)


,i,t,v_x_1,v_x_2,v_x_3,v_p
0,1,1,0.222337,-0.505634,0.455673,1.069062
1,1,2,0.488172,-1.162879,-1.162611,0.052533
2,1,3,0.568830,1.252223,-0.197146,1.563581
3,1,4,0.781116,-0.655212,0.743714,0.805346
4,1,5,0.933814,0.819826,-1.413157,-0.051522


## 1.5 df
Join $X$, $M$, $V$ and name it df. df is the data frame such that a row contains variables for a consumer about a product that is available in a market.

In [9]:
df = M.merge(X, how = 'left')
df = df.merge(V, how = 'left')
df.sort_values(by = ['t', 'i', 'j'], inplace =True)

print('shape of df: ', df.shape)
df.head()

shape of df:  (12600, 13)


,j,t,xi,c,p,x_2,x_3,x_1,i,v_x_1,v_x_2,v_x_3,v_p
10600,0.0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0.222337,-0.505634,0.455673,1.069062
10500,5.0,1,0.140941,1.001834,2.141619,4.584802,3.153910,1,1,0.222337,-0.505634,0.455673,1.069062
10400,6.0,1,-0.090019,1.018559,1.896693,4.112289,1.729779,1,1,0.222337,-0.505634,0.455673,1.069062
10601,0.0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,1.217603,0.344503,0.116888,-0.073015
10501,5.0,1,0.140941,1.001834,2.141619,4.584802,3.153910,1,2,1.217603,0.344503,0.116888,-0.073015


## 1.6 e
Draw a vector of preference shocks e whose length is the same as the number of rows of df.

In [10]:
e = np.random.gumbel(size = (len(df), 1))

print('shape of e: ', e.shape)

e

shape of e:  (12600, 1)


array([[ 0.16418068],
       [-0.81040583],
       [-0.55379447],
       ...,
       [ 1.45825932],
       [-1.30366683],
       [ 3.45069786]])

## 1.7 compute indirect utility (df, beta, sigma, mu, omega)
Write a function compute_indirect_utility(df, beta, sigma, mu, omega) that returns a vector whose element is the mean indirect utility of a product for a consumer in a market. The output should have the same length with e.

The indirect utility of consumer $i$ in market $t$ for product $j$ is: $$u_{itj}=\beta_{it}'x_j + \alpha_{it}p_{jt} +\xi_{jt} + \epsilon_{ijt}$$

$\beta_{it}$ and $\alpha_{it}$ are different across consumers, and they are distributed as:
- $\beta_{itk} = \beta_{0k} + \sigma_k v_{itk},$ where $v_{itk}$ for $k=1,2,\cdots, K$ are i.i.d. standard normal random variables.
  
- $\alpha_{it} = -exp(\mu + \omega v_{it}) = -exp(\mu + \omega^2/2) + [-exp(\mu + \omega v_{it}) + exp(\mu + \omega^2/2)] = \alpha_0 + \tilde{\alpha}_{it},$ where $v_{it}$ are i.i.d. standard normal random variables.
  - $[\beta_{01}, \cdots, \beta_{0K}]$ is referred to as beta.
  - $[\sigma_{1}, \cdots, \sigma_{K}]$ is referred to as sigma.

In [11]:
# compute indirect utility (df, beta, sigma, mu, omega)
def compute_indirect_utility(df, beta, sigma, mu, omega):
    u = (beta[0]+sigma[0]*df.v_x_1)*df.x_1 + (beta[1]+sigma[1]*df.v_x_2)*df.x_2 + (beta[2]+sigma[2]*df.v_x_3)*df.x_3 - \
    np.exp(mu + omega*df.v_p)*df.p + \
    df.xi
    return u

In [12]:
u = compute_indirect_utility(df, beta, sigma, mu, omega)

print('shape of u: ', u.shape)

u

shape of u:  (12600,)


10600     0.000000
10500   -10.723310
10400    -9.090499
10601     0.000000
10501     0.104487
           ...    
97       -2.967514
12598     0.000000
98        1.791335
12599     0.000000
99      -10.745317
Length: 12600, dtype: float64

## 1.8 compute_choice_smooth(df, e, beta, sigma, mu, omega)
In Assingment 3, we computed predicted share by simulating choice and taking their average. Instead, we compute the actual share by:
$$s_{jt}=\frac{1}{N}\sum_{i=1}^N \frac{exp(\beta_{it}' x_{j} + \alpha_{it}p_{jt} + \xi_{jt})}{1 + \sum_{k\in J_t}exp(\beta_{it}' x_{k} + \alpha_{it}p_{kt} + \xi_{kt})}$$
and the predicted share by:
$$\hat{\sigma}_{jt}=\frac{1}{L}\sum_{l=1}^L \frac{exp(\beta_{t}^{(l)'} x_{j} + \alpha_{t}^{(l)}p_{jt} + \xi_{jt})}{1 + \sum_{k\in J_t}exp(\beta_{t}^{(l)'} x_{k} + \alpha_{t}^{(l)}p_{kt} + \xi_{kt})}$$

To do so, write a function compute_choice_smooth(X, M, V, beta, sigma, mu, omega) in which the choice of each consumer is not: $$q_{ijt}=1\{u_{ijt} = \underset{k\in J_{t}\cup \{0\}}{max} u_{ikt}\}.$$
but $$\tilde{q}_{ijt}=\frac{exp(u_{ijt})}{\sum_{k\in J_t}} u_{ikt}$$

In [13]:
def compute_choice_smooth(df, beta, sigma, mu, omega):
    df['u'] = compute_indirect_utility(df, beta, sigma, mu, omega)
    df['exp_u'] = np.exp(df['u'])
    
    temp = df.groupby(['t']).agg({'exp_u':'sum'}).reset_index()
    temp.columns = ['t', 'sum_exp_u']
    df = df.merge(temp, how = 'left')
    df['q'] = df['exp_u']/df['sum_exp_u']
    df.drop(columns = ['exp_u', 'sum_exp_u'], inplace = True)
    return df

In [14]:
df_choice_smooth = compute_choice_smooth(df, beta, sigma, mu, omega)

print('shape of df_choice_smooth: ', df_choice_smooth.shape)
df_choice_smooth.describe()

shape of df_choice_smooth:  (12600, 15)


,j,t,xi,c,p,x_2,x_3,x_1,i,v_x_1,v_x_2,v_x_3,v_p,u,q
count,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,12600.000000,1.260000e+04
mean,4.714286,10.412698,0.039806,0.849154,1.882130,1.464391,0.573792,0.841270,50.500000,0.006233,-0.036729,-0.021411,0.019572,-2.908460,1.587302e-03
std,2.844928,5.505295,0.488794,0.371346,1.117639,2.418014,2.518783,0.365439,28.867216,1.001965,1.014278,0.998345,1.032963,9.501122,2.874954e-02
min,0.000000,1.000000,-1.284838,0.000000,0.000000,-1.954350,-5.254284,0.000000,1.000000,-3.633006,-3.117845,-3.239084,-4.123845,-214.727202,5.855860e-101
25%,2.000000,6.000000,-0.197126,0.955844,1.546005,0.000000,-0.235369,1.000000,25.750000,-0.685562,-0.759284,-0.704411,-0.675947,-5.395780,7.469515e-11
50%,5.000000,11.000000,0.000000,0.996053,1.960644,0.507840,1.638937,1.000000,50.500000,0.006309,0.006259,-0.030419,0.022255,-0.021987,8.145437e-09
75%,7.000000,15.000000,0.292292,1.028906,2.351592,4.112289,1.729779,1.000000,75.250000,0.688782,0.657977,0.681589,0.714855,1.496124,2.058947e-07
max,10.000000,20.000000,1.650833,1.129210,6.398996,6.444651,3.493566,1.000000,100.000000,2.849756,3.402405,2.749063,3.208216,21.111913,9.737987e-01


## 1.9 compute_share(df, e, beta, sigma, mu, omega)
Next, write a function compute_share_smooth(X, M, V, beta, sigma, mu, omega) that calls compute_choice_smooth and then returns the share based on above $\tilde{q}_{ijt}$. If we use these functions with the Monte Carlo shocks, it gives us the predicted share of the products.

The share: $$s_{jt}=\frac{1}{N}\sum_{i=1}^N q_{ijt}$$

In [15]:
def compute_share_smooth(df, e, beta, sigma, mu, omega):
    df_choice_smooth = compute_choice_smooth(df, beta, sigma, mu, omega)
    
    temp1 = df_choice_smooth.groupby(['j', 't', 'x_1', 'x_2', 'x_3',  'xi', 'c', 'p']).agg({'q': 'mean'}).reset_index()
    temp1.columns = ['j', 't', 'x_1', 'x_2', 'x_3',  'xi', 'c', 'p', 's']

    temp2 = temp1.loc[temp1.j == 0, ['t','s']]
    temp2.columns = ['t', 's_0']
    
    temp1 = temp1.merge(temp2, how = 'left')
    temp1['y'] = np.log(temp1['s']) - np.log(temp1['s_0'])
    return temp1[['j', 't', 'x_2', 'x_3', 'x_1', 'xi', 'c', 'p', 's', 'y']]

In [16]:
df_share_smooth = compute_share_smooth(df, e, beta, sigma, mu, omega)

print('shape of df_share_smooth: ', df_share_smooth.shape)
df_share_smooth.tail()

shape of df_share_smooth:  (126, 10)


,j,t,x_2,x_3,x_1,xi,c,p,s,y
121,9.0,12,6.444651,1.127247,1,0.344830,1.032754,2.517657,1.667600e-03,10.175986
122,9.0,15,6.444651,1.127247,1,-0.415538,1.011631,1.653603,5.523013e-04,11.008259
123,10.0,2,-0.274661,-0.582359,1,-0.443547,0.979209,1.629343,1.121272e-07,2.622176
124,10.0,3,-0.274661,-0.582359,1,0.299507,1.129210,2.480437,2.269019e-06,3.070286
125,10.0,12,-0.274661,-0.582359,1,0.194604,1.034812,2.208121,9.052079e-07,2.657255


# 2  Estimate the parameters

## 2.1
First draw Monte Carlo consumer-level heterogeneity V_mcmc and Monte Carlo preference shocks e_mcmc. The number of simulations is L. This does not have to be the same with the actual number of consumers N.

In [17]:
# The number of simulations is L

V_mcmc = pd.DataFrame([(i+1,1) for i in range(L)], columns = ['i', 'temp'])\
.merge(pd.DataFrame([(t+1,1) for t in range(T)], columns = ['t', 'temp']), how = 'outer')
V_mcmc.drop(columns = 'temp', inplace =True)
for col in ['v_x_1', 'v_x_2', 'v_x_3', 'v_p']:
    V_mcmc[col] = np.random.normal(size = (L*T, 1))

print('shape of V_mcmc: ', V_mcmc.shape)
V_mcmc.head()

shape of V_mcmc:  (2000, 6)


,i,t,v_x_1,v_x_2,v_x_3,v_p
0,1,1,1.016342,-1.076901,-0.946140,-0.562902
1,1,2,0.611142,0.437597,1.485381,-0.713417
2,1,3,-0.336947,0.563302,0.320777,0.418745
3,1,4,1.025688,-0.836516,1.721432,0.938075
4,1,5,0.606180,0.942697,-1.922313,1.482613


In [18]:
df_mcmc = M.merge(X, how = 'left')
df_mcmc = df_mcmc.merge(V_mcmc, how = 'left')

print('shape of df_mcmc: ', df_mcmc.shape)
df_mcmc.head()

shape of df_mcmc:  (12600, 13)


,j,t,xi,c,p,x_2,x_3,x_1,i,v_x_1,v_x_2,v_x_3,v_p
0,6.0,20,-0.83495,1.050409,1.485568,4.112289,1.729779,1,1,0.839740,-0.688602,-0.307306,0.435724
1,6.0,20,-0.83495,1.050409,1.485568,4.112289,1.729779,1,2,-0.111596,-0.569594,1.906566,-0.032177
2,6.0,20,-0.83495,1.050409,1.485568,4.112289,1.729779,1,3,-0.934448,0.510063,1.688907,-0.552010
3,6.0,20,-0.83495,1.050409,1.485568,4.112289,1.729779,1,4,0.697681,0.207414,-0.901238,-1.032272
4,6.0,20,-0.83495,1.050409,1.485568,4.112289,1.729779,1,5,-0.315217,-0.517918,0.147684,1.170911


In [19]:
e_mcmc = np.random.gumbel(size = (len(df_mcmc), 1))

print('shape of e_mcmc: ', e_mcmc.shape)
e_mcmc

shape of e_mcmc:  (12600, 1)


array([[-0.22111719],
       [ 0.77280605],
       [-0.04096708],
       ...,
       [ 2.20960242],
       [ 2.02961705],
       [ 1.42053535]])

## 2.2
Vectorize the parameters to a vector theta.

In [20]:
# set parameters
theta = beta.flatten().tolist()

for item in sigma.flatten().tolist():
    theta.append(item)
    
theta.append(mu)
theta.append(omega)

theta

[4.0,
 -0.3471288055213315,
 -0.6439357431511162,
 0.6230721565147973,
 1.0996384047171581,
 1.0111807889525577,
 0.5,
 1]

## 2.3 
Estimate the parameters assuming there is no product-specific unobserved fixed effects $\xi_{jt}$, i.e., using the functions in assignment 3. To do so, first modify M to M_no in which xi is replaced with 0 and estimate the model with M_no. Otherwise, your function will compute the share with the true xi.

In [21]:
M_no = M.copy()
M_no['xi'] = 0

print('shape of M_no: ', M_no.shape)
M_no.head()

shape of M_no:  (126, 5)


,j,t,xi,c,p
119,6.0,20,0,1.050409,1.485568
118,6.0,19,0,0.953264,2.764732
98,5.0,19,0,0.957834,2.281137
158,8.0,19,0,1.072073,2.085452
138,7.0,19,0,1.069503,1.638399


In [22]:
# minimize objective

Next, we estimate the model allowing for the product-market-specific unobserved fixed effect $\xi_{jt}$ using the BLP algorithm. To do so, we slightly modify the compute_indirect_utility, compute_choice_smooth, and compute_share_smooth functions so that they receive $\delta_{jt}$ to compute the indirect utilities, choices, and shares. Be careful that the treatment of $\alpha_i$ is slightly different from the lecture note, because we assumed that $\alpha_i$s are log-normal random variables.

## 2.4 Compute and print out $\delta_{jt}$  at the true parameters, i.e.
$$\delta_{jt}= \beta_0'x_j + \alpha_0'p_{jt}+\xi_{jt}$$

In [23]:
def calculate_delta(X, M, beta, mu, omega):
    delta = M.merge(X, how = 'left')
    delta['delta'] = (beta[0])*delta.x_1 + (beta[1])*delta.x_2 + (beta[2])*delta.x_3 - np.exp(mu+np.power(omega,2)/2)*delta.p + delta.xi
    return delta[['j', 't', 'delta']]

In [24]:
delta = calculate_delta(X, M, beta, mu, omega)
delta

,j,t,delta
0,6.0,20,-3.414504
1,6.0,19,-5.498663
2,5.0,19,-5.430234
3,8.0,19,1.765972
4,7.0,19,-1.401028
...,...,...,...
121,0.0,16,0.000000
122,0.0,17,0.000000
123,0.0,18,0.000000
124,0.0,19,0.000000


## 2.5  compute_indirect_utility_delta(df, delta, sigma, mu, omega)
Write a function compute_indirect_utility_delta(df, delta, sigma, mu, omega) that returns a vector whose element is the mean indirect utility of a product for a consumer in a market. The output should have the same length with $e$
. Print out the output with  $\delta_{jt}$ evaluated at the true parameters. Check if the output is close to the true indirect utilities.

In [25]:
@jit(nopython = False, parallel = True) 
def compute_indirect_utility_delta(df, delta, sigma, mu, omega):
    df = df.merge(delta, how = 'left')
    u = df.delta + (sigma[0]*df.v_x_1)*df.x_1 + (sigma[1]*df.v_x_2)*df.x_2 + (sigma[2]*df.v_x_3)*df.x_3\
    - np.exp(mu + omega*df.v_p)*df.p + np.exp(mu+np.power(omega,2)/2)*df.p
    return u

In [26]:
u_delta = compute_indirect_utility_delta(df_mcmc, delta, sigma, mu, omega)

u_delta

<ipython-input-25-a97cf36eda7f>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "compute_indirect_utility_delta" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-25-a97cf36eda7f> (3)

File "<ipython-input-25-a97cf36eda7f>", line 3:
def compute_indirect_utility_delta(df, delta, sigma, mu, omega):
    df = df.merge(delta, how = 'left')
    ^

  @jit(nopython = False, parallel = True)
/opt/anaconda3/lib/python3.8/site-packages/numba/core/object_mode_passes.py:177: NumbaWarning: Function "compute_indirect_utility_delta" was compiled in object mode without forceobj=True.

File "<ipython-input-25-a97cf36eda7f>", line 2:
@jit(nopython = False, parallel = True) 
def compute_indirect_utility_delta(df, delta, sigma, mu, omega):
^

  warnings.warn(errors.NumbaWarning(warn_msg,


0       -6.291289
1       -1.058485
2        3.891803
3       -0.452473
4       -9.555223
           ...   
12595    0.000000
12596    0.000000
12597    0.000000
12598    0.000000
12599    0.000000
Length: 12600, dtype: float64

In [27]:
u = compute_indirect_utility(df_mcmc, beta, sigma, mu, omega)
(u - u_delta).describe()

count    1.260000e+04
mean     7.972899e-17
std      1.236576e-15
min     -1.421085e-14
25%      0.000000e+00
50%      0.000000e+00
75%      4.440892e-16
max      1.421085e-14
dtype: float64

## 2.6 compute_choice_smooth_delta(X, M, V, delta, sigma, mu, omega)
Write a function compute_choice_smooth_delta(X, M, V, delta, sigma, mu, omega) that first construct df from X, M, V, second call compute_indirect_utility_delta to obtain the vector of mean indirect utilities u, third compute the (smooth) choice vector q based on the vector of mean indirect utilities, and finally return the data frame to which u and q are added as columns. Print out the output with $\delta_{jt}$ evaluated at the true parameters. Check if the output is close to the true (smooth) choice vector.

In [28]:
@jit(nopython = False, parallel = True) 
def compute_choice_smooth_delta(df, delta, sigma, mu, omega):
    df['u'] = compute_indirect_utility_delta(df, delta, sigma, mu, omega)
    df['exp_u'] = np.exp(df['u'])
    
    temp = df.groupby(['t']).agg({'exp_u':'sum'}).reset_index()
    temp.columns = ['t', 'sum_exp_u']
    df = df.merge(temp, how = 'left')
    df['q'] = df['exp_u']/df['sum_exp_u']
    df.drop(columns = ['exp_u', 'sum_exp_u'], inplace = True)
    return df

## 2.7 compute_share_delta(df, delta, sigma, mu, omega)

In [29]:
@jit(nopython = False, parallel = True) 
def compute_share_smooth_delta(df, delta, sigma, mu, omega):
    df_choice_smooth_delta = compute_choice_smooth_delta(df, delta, sigma, mu, omega)
    
    temp1 = df_choice_smooth_delta.groupby(['j', 't', 'x_1', 'x_2', 'x_3',  'xi', 'c', 'p']).agg({'q': 'mean'}).reset_index()
    temp1.columns = ['j', 't', 'x_1', 'x_2', 'x_3',  'xi', 'c', 'p', 's']

    temp2 = temp1.loc[temp1.j == 0, ['t','s']]
    temp2.columns = ['t', 's_0']
    
    temp1 = temp1.merge(temp2, how = 'left')
    temp1['y'] = np.log(temp1['s']) - np.log(temp1['s_0'])
    return temp1[['j', 't', 'x_2', 'x_3', 'x_1', 'xi', 'c', 'p', 's', 'y']]

In [30]:
df_share_smooth_delta = compute_share_smooth_delta(df, delta, sigma, mu, omega)

print(df_share_smooth_delta.shape)
df_share_smooth_delta.head()

<ipython-input-29-ebc11e926bc4>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "compute_share_smooth_delta" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-29-ebc11e926bc4> (3)

File "<ipython-input-29-ebc11e926bc4>", line 3:
def compute_share_smooth_delta(df, delta, sigma, mu, omega):
    df_choice_smooth_delta = compute_choice_smooth_delta(df, delta, sigma, mu, omega)
    ^

  @jit(nopython = False, parallel = True)
/opt/anaconda3/lib/python3.8/site-packages/numba/core/object_mode_passes.py:177: NumbaWarning: Function "compute_share_smooth_delta" was compiled in object mode without forceobj=True.

File "<ipython-input-29-ebc11e926bc4>", line 2:
@jit(nopython = False, parallel = True) 
def compute_share_smooth_delta(df, delta, sigma, mu, omega):
^

  warnings.warn(errors.NumbaWarning(warn_msg,
<ipython-input-28-42376038c459>:1: NumbaWarning: 
Compilation is falling back to obje

(126, 10)


/opt/anaconda3/lib/python3.8/site-packages/numba/core/object_mode_passes.py:177: NumbaWarning: Function "compute_choice_smooth_delta" was compiled in object mode without forceobj=True.

File "<ipython-input-28-42376038c459>", line 2:
@jit(nopython = False, parallel = True) 
def compute_choice_smooth_delta(df, delta, sigma, mu, omega):
^

  warnings.warn(errors.NumbaWarning(warn_msg,


,j,t,x_2,x_3,x_1,xi,c,p,s,y
0,0.0,1,0.0,0.0,0,0.0,0.0,0.0,0.000317,0.0
1,0.0,2,0.0,0.0,0,0.0,0.0,0.0,0.002317,0.0
2,0.0,3,0.0,0.0,0,0.0,0.0,0.0,0.000006,0.0
3,0.0,4,0.0,0.0,0,0.0,0.0,0.0,0.000310,0.0
4,0.0,5,0.0,0.0,0,0.0,0.0,0.0,0.000262,0.0


## 2.8 solve_delta(share, X, M, V, delta_initial, sigma, mu, omega)
Write a function solve_delta(share, X, M, V, delta, sigma, mu, omega) that finds $\delta_{jt}$ that equates the actua share and the predicted share based on compute_share_smooth_delta by the fixed-point algorithm with an operator:


In [31]:
@jit(nopython = False, parallel = True) 
def solve_delta(share, X, M, V, delta_initial, sigma, mu, omega, kappa, lam):
    df = M.merge(X, how = 'left')
    df = df.merge(V, how = 'left')
    delta = delta_initial.copy()
    eps = 10
    while eps >= lam:
        df_share_smooth_delta_old = compute_share_smooth_delta(df, delta, sigma, mu, omega)
        delta_old_vector = delta.delta
        delta['delta'] = delta.delta + kappa*np.log(share['s']/df_share_smooth_delta_old['s'])
        delta.loc[delta.j == 0, 'delta'] = 0
        eps = np.max(np.abs(delta_old_vector - delta.delta))
    return delta

In [32]:
kappa = 1
lam = 1e-3
delta_initial = calculate_delta(X, M, beta, mu, omega)
share = compute_share_smooth_delta(df, delta_initial, sigma, mu, omega)[['j', 't', 's']]
delta_new = solve_delta(share, X, M, V_mcmc, delta_initial, sigma, mu, omega, kappa, lam)

<ipython-input-31-298657ebb171>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "solve_delta" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-31-298657ebb171> (3)

File "<ipython-input-31-298657ebb171>", line 3:
def solve_delta(share, X, M, V, delta_initial, sigma, mu, omega, kappa, lam):
    df = M.merge(X, how = 'left')
    ^

  @jit(nopython = False, parallel = True)
/opt/anaconda3/lib/python3.8/site-packages/numba/core/object_mode_passes.py:177: NumbaWarning: Function "solve_delta" was compiled in object mode without forceobj=True.

File "<ipython-input-31-298657ebb171>", line 2:
@jit(nopython = False, parallel = True) 
def solve_delta(share, X, M, V, delta_initial, sigma, mu, omega, kappa, lam):
^

  warnings.warn(errors.NumbaWarning(warn_msg,


We use the marginal cost $c_{jt}$ as the excluded instrumental variable for $p_{jt}$. Let $\Psi$ be the weighing matrix for the GMM estimator. For now, let it be the identity matrix. Write a function compute_theta_linear(df_share_smooth, delta, mu, omega, Psi) that returns the optimal linear parameters associated with the data and $\delta$. Notice that we only obtain $\beta_0$ in this way because $\alpha_0$ is directly computed from the non-linear parameters by $-exp(\mu + \omega^2/2)$. The first order condition for $\beta_0$ is:

Recall IV estimator as $\hat{{\beta}}_{IV}=(R'W'X)^{-1}R'W'Y$
- $R = (ZW^{-1})' X $

Thus,
$$(((ZW^{-1})' X)' Z' X)^{-1} ((ZW^{-1})' X)' Z' Y = ((Z'X)'W^{-1} Z'X)^{-1} (Z'X)'W^{-1} Z'Y $$

$$\delta_{jt}= \beta_{01}'x_1 + \beta_{02}'x_2 + \beta_{03}'x_3 + \alpha_0'p_{jt}+\xi_{jt}$$

In [33]:
df_share_smooth

,j,t,x_2,x_3,x_1,xi,c,p,s,y
0,0.0,1,0.000000,0.000000,0,0.000000,0.000000,0.000000,1.654077e-06,0.000000
1,0.0,2,0.000000,0.000000,0,0.000000,0.000000,0.000000,8.145437e-09,0.000000
2,0.0,3,0.000000,0.000000,0,0.000000,0.000000,0.000000,1.053003e-07,0.000000
3,0.0,4,0.000000,0.000000,0,0.000000,0.000000,0.000000,1.654328e-05,0.000000
4,0.0,5,0.000000,0.000000,0,0.000000,0.000000,0.000000,2.423467e-05,0.000000
...,...,...,...,...,...,...,...,...,...,...
121,9.0,12,6.444651,1.127247,1,0.344830,1.032754,2.517657,1.667600e-03,10.175986
122,9.0,15,6.444651,1.127247,1,-0.415538,1.011631,1.653603,5.523013e-04,11.008259
123,10.0,2,-0.274661,-0.582359,1,-0.443547,0.979209,1.629343,1.121272e-07,2.622176
124,10.0,3,-0.274661,-0.582359,1,0.299507,1.129210,2.480437,2.269019e-06,3.070286


In [34]:
def compute_theta_linear(df_share_smooth, delta, Psi, mu, omega):
    alpha_0 = -np.exp(mu+np.power(omega,2)/2)
    df_share_smooth = df_share_smooth.merge(delta, how = 'left')
    Psi_inv = np.linalg.inv(Psi)
    X_temp = df_share_smooth[df_share_smooth.j != 0][['x_1', 'x_2', 'x_3']].values 
    Z_temp = df_share_smooth[df_share_smooth.j != 0][['x_1', 'x_2', 'x_3', 'c']].values 
    Y = df_share_smooth[df_share_smooth.j != 0][['delta']].values - alpha_0 * df_share_smooth[df_share_smooth.j != 0][['p']].values
    ZX = Z_temp.T @ X_temp
    beta_hat = np.linalg.inv(ZX.T @ Psi_inv @ ZX) @ ZX.T @ Psi_inv @ Z_temp.T @ Y
    return beta_hat

In [35]:
Psi = np.identity(len(beta)+1)
compute_theta_linear(df_share_smooth, delta, Psi, mu, omega)

array([[ 4.04184147],
       [-0.34320557],
       [-0.6468904 ]])

In [36]:
beta

array([[ 4.        ],
       [-0.34712881],
       [-0.64393574]])

## solve_xi(df_share_smooth, delta, beta, mu, omega)

In [37]:
def solve_xi(df_share_smooth, delta, beta, mu, omega):
    df_share_smooth = df_share_smooth.merge(delta, how = 'left')
    df_share_smooth = df_share_smooth[df_share_smooth.j != 0]
    xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
    return xi

In [38]:
xi = solve_xi(df_share_smooth, delta, beta, mu, omega)
xi

20    -0.869627
21    -0.167289
22    -0.208360
23     0.228257
24    -0.115748
         ...   
121    0.344830
122   -0.415538
123   -0.443547
124    0.299507
125    0.194604
Length: 106, dtype: float64

In [39]:
theta_nonlinear = list()
    
theta_nonlinear.append(mu)
theta_nonlinear.append(omega)

for item in sigma.flatten().tolist():
    theta_nonlinear.append(item)
    
theta_nonlinear

[0.5, 1, 0.6230721565147973, 1.0996384047171581, 1.0111807889525577]

In [40]:
def GMM_objective_A4(theta_nonlinear, delta, df_share_smooth, Psi, X, M, V_mcmc, kappa, lam):
    mu = theta_nonlinear[0]
    omega = theta_nonlinear[1]
    sigma = theta_nonlinear[2:]
    share = df_share_smooth[['j', 't', 's']]
    Z_temp = df_share_smooth[df_share_smooth.j != 0][['x_1', 'x_2', 'x_3', 'c']].values 
    Psi_inv = np.linalg.inv(Psi)
    delta_converge = solve_delta(share, X, M, V_mcmc, delta, sigma, mu, omega, kappa, lam)
    beta_hat = compute_theta_linear(df_share_smooth, delta_converge, Psi, mu, omega)
    xi = np.array(solve_xi(df_share_smooth, delta, beta_hat, mu, omega))
    obj = xi.T @ Z_temp @ Psi_inv @ Z_temp.T @ xi
    return obj

In [41]:
GMM_objective_A4(theta_nonlinear, delta, df_share_smooth, Psi, X, M, V_mcmc, kappa, lam)

<ipython-input-31-298657ebb171>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "solve_delta" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-31-298657ebb171> (3)

File "<ipython-input-31-298657ebb171>", line 3:
def solve_delta(share, X, M, V, delta_initial, sigma, mu, omega, kappa, lam):
    df = M.merge(X, how = 'left')
    ^

  @jit(nopython = False, parallel = True)
/opt/anaconda3/lib/python3.8/site-packages/numba/core/object_mode_passes.py:177: NumbaWarning: Function "solve_delta" was compiled in object mode without forceobj=True.

File "<ipython-input-31-298657ebb171>", line 2:
@jit(nopython = False, parallel = True) 
def solve_delta(share, X, M, V, delta_initial, sigma, mu, omega, kappa, lam):
^

  warnings.warn(errors.NumbaWarning(warn_msg,
<ipython-input-29-ebc11e926bc4>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Functio

70714.12659532543

In [42]:
theta_nonlinear_initial = [0.5, 0.5, 0.5, 0.5, 0.5]
res = minimize(GMM_objective_A4, theta_nonlinear_initial,  
               args = (delta, df_share_smooth, Psi, X, M, V_mcmc, 
                      kappa, lam),
               tol=1e-6)
print(res.x)

<ipython-input-31-298657ebb171>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "solve_delta" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-31-298657ebb171> (3)

File "<ipython-input-31-298657ebb171>", line 3:
def solve_delta(share, X, M, V, delta_initial, sigma, mu, omega, kappa, lam):
    df = M.merge(X, how = 'left')
    ^

  @jit(nopython = False, parallel = True)
/opt/anaconda3/lib/python3.8/site-packages/numba/core/object_mode_passes.py:177: NumbaWarning: Function "solve_delta" was compiled in object mode without forceobj=True.

File "<ipython-input-31-298657ebb171>", line 2:
@jit(nopython = False, parallel = True) 
def solve_delta(share, X, M, V, delta_initial, sigma, mu, omega, kappa, lam):
^

  warnings.warn(errors.NumbaWarning(warn_msg,
<ipython-input-29-ebc11e926bc4>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Functio

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X, M, V_mcmc, delta, sigma, mu, omega, kappa, lam)
<ipython-input-34-cd52e929264d>:2: RuntimeWarning: overflow encountered in exp
  alpha_0 = -np.exp(mu+np.power(omega,2)/2)
<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X

<ipython-input-34-cd52e929264d>:2: RuntimeWarning: overflow encountered in exp
  alpha_0 = -np.exp(mu+np.power(omega,2)/2)
<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X, M, V_mcmc, delta, sigma, mu, omega, kappa, lam)
<ipython-input-34-cd52e929264d>:2: RuntimeWarning: overflow encountered in exp
  alpha_0 = -np.exp(mu+np.power(omega,2)/2)
<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_

<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X, M, V_mcmc, delta, sigma, mu, omega, kappa, lam)
<ipython-input-34-cd52e929264d>:2: RuntimeWarning: overflow encountered in exp
  alpha_0 = -np.exp(mu+np.power(omega,2)/2)
<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X, M, V_mcmc, delta, sigma, mu, omega, kappa, lam)
<ipython-input-34-cd52e929264d>:2: RuntimeWarning: overflow encountered in exp
  alpha_0 = -np.exp(mu+np.power(omega,2)/2)
<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X

...

<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X, M, V_mcmc, delta, sigma, mu, omega, kappa, lam)
<ipython-input-34-cd52e929264d>:2: RuntimeWarning: overflow encountered in exp
  alpha_0 = -np.exp(mu+np.power(omega,2)/2)
<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8

<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X, M, V_mcmc, delta, sigma, mu, omega, kappa, lam)
<ipython-input-34-cd52e929264d>:2: RuntimeWarning: overflow encountered in exp
  alpha_0 = -np.exp(mu+np.power(omega,2)/2)
<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8

<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X, M, V_mcmc, delta, sigma, mu, omega, kappa, lam)
<ipython-input-34-cd52e929264d>:2: RuntimeWarning: overflow encountered in exp
  alpha_0 = -np.exp(mu+np.power(omega,2)/2)
<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8

[ 11331.28531577 -22883.76476111 -11910.67719567  81817.41599118
  25518.3759235 ]


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X, M, V_mcmc, delta, sigma, mu, omega, kappa, lam)
<ipython-input-34-cd52e929264d>:2: RuntimeWarning: overflow encountered in exp
  alpha_0 = -np.exp(mu+np.power(omega,2)/2)
<ipython-input-37-f16c32cbe0b2>:4: RuntimeWarning: overflow encountered in exp
  xi = df_share_smooth.delta - ((beta[0])*df_share_smooth.x_1 + (beta[1])*df_share_smooth.x_2 + (beta[2])*df_share_smooth.x_3 - np.exp(mu+np.power(omega,2)/2)*df_share_smooth.p)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-40-a44aec5732c5>:8: RuntimeWarning: overflow encountered in exp
  delta_converge = solve_delta(share, X